In [ ]:
import numpy as np
from scipy.spatial import cKDTree

from mmtwfs.wfs import WFSFactory, aperture_distance
from mmtwfs.zernike import (
    zernike_slopes
)

In [ ]:
zv = {
    "Z04": 500.,
    "Z05": 1000.,
    "Z06": 1500.,
    "Z11": 1000.,
    "Z22": 500.,
    "Z33": 100.,
}
r = np.linspace(0.1, 1, 100)
theta = np.linspace(0, 2*np.pi, 145)
rho, phi = np.meshgrid(r, theta)

In [ ]:
%timeit zernike_slopes(zv, rho, phi, norm=True, use_cache=True)

In [ ]:
%timeit zernike_slopes(zv, rho, phi, norm=True, use_cache=False)

In [ ]:
mmirs = WFSFactory(wfs="mmirs")
mmirs_file = "/home/tim/MMT/mmtwfs/mmtwfs/data/test_data/mmirs_wfs_0150.fits"

In [ ]:
%%timeit
results = mmirs.measure_slopes(mmirs_file, plot=False)
zresults = mmirs.fit_wavefront(results, plot=False)
#results['seeing']

In [ ]:
%%prun
results = mmirs.measure_slopes(mmirs_file, plot=False)
zresults = mmirs.fit_wavefront(results, plot=False)

In [ ]:
x = np.arange(250)
y = np.arange(250)
rx = np.arange(300) + 10
ry = np.arange(300) + 3
a1 = np.array([x, y])
a2 = np.array([rx, ry])
ad = aperture_distance(x, y, rx, ry)
ad

In [ ]:
def aperture_distance(refx, refy, spotx, spoty):
    """
    Calculate the sum of the distances between each reference aperture and the closest measured spot position.
    This total distance is the statistic to minimize when fitting the reference aperture grid to the data.
    """
    tot_dist = 0.0
    refs = np.array([refx, refy]).transpose()
    spots = np.array([spotx, spoty]).transpose()
    tree = cKDTree(refs)
    mindist, _ = tree.query(spots)
    tot_dist = mindist.sum()
    return np.log(tot_dist)

In [ ]:
%%timeit
d = aperture_distance(rx, ry, x, y)

In [ ]:
refs = np.array([rx, ry]).transpose()
spots = np.array([x, y]).transpose()

In [ ]:
tree = cKDTree(refs)

In [ ]:
mindist, minid = tree.query(spots)

In [ ]:
np.log(mindist.sum())

In [ ]:
a = np.arange(25).reshape(5, 5)

In [ ]:
a.sum(axis=0)

In [ ]:
a.sum(axis=1)

In [ ]:
a